In [0]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
import category_encoders as ce
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib


In [0]:
# Define the relative path to the Dataset folder
dataset_path = '../Datasets/df_aggregated_month_populated_btp_core.parquet'

# Read DataFrame from Parquet
df_aggregated_month_populated_btp_core = pd.read_parquet(dataset_path, engine='pyarrow')
print(df_aggregated_month_populated_btp_core)

In [0]:
df_aggregated_month_populated_btp_core.columns

In [0]:
# Custom MAPE function that ignores zero values in actuals (y_true)
def mean_absolute_percentage_error(y_true, y_pred):
    mask = y_true != 0  # Ignore zero values
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

In [0]:
# First test without horizons
# Step 1: Define the features and target variable
all_features = [
    'MONTHLY_CONTRACT_NET_VALUE_SUM',  
    'CONTRACT_DURATION_SUM', 
    'OVERCONSUMPTION_COUNT', 
    'ORDER_COUNT', 
    'ACTIVE_CONTRACT', 
    'TOTAL_CONSUMPTION_LAG_1'
]

X = df_aggregated_month_populated_btp_core[all_features].copy()
y = df_aggregated_month_populated_btp_core['TOTAL_CONSUMPTION_SUM']

# Step 2: Sort the data by date
df_aggregated_month_populated_btp_core.sort_values(by='DATE', inplace=True)

# Step 3: Set up TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Initialize an empty dictionary to store evaluation metrics
feature_performance_tscv = {}

# Loop to progressively remove features and evaluate performance
while len(all_features) > 0:
    print(f"Using features: {all_features}")
    X_subset = X[all_features]
    
    # Initialize lists to store evaluation metrics for each split
    mse_train_list, mse_test_list = [], []
    r2_train_list, r2_test_list = [], []
    mae_train_list, mae_test_list = [], []
    mape_train_list, mape_test_list = [], []

    # Perform time series cross-validation
    for train_index, test_index in tscv.split(X_subset):
        X_train, X_test = X_subset.iloc[train_index], X_subset.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the Linear Regression model
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Make predictions
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        
        # Calculate evaluation metrics
        mse_train = mean_squared_error(y_train, y_train_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        
        mae_train = mean_absolute_error(y_train, y_train_pred)
        mae_test = mean_absolute_error(y_test, y_test_pred)
        
        # Calculate MAPE while ignoring zero values
        mape_train = np.mean(np.abs((y_train[y_train != 0] - y_train_pred[y_train != 0]) / y_train[y_train != 0])) * 100
        mape_test = np.mean(np.abs((y_test[y_test != 0] - y_test_pred[y_test != 0]) / y_test[y_test != 0])) * 100
        
        # Store the metrics for each fold
        mse_train_list.append(mse_train)
        mse_test_list.append(mse_test)
        r2_train_list.append(r2_train)
        r2_test_list.append(r2_test)
        mae_train_list.append(mae_train)
        mae_test_list.append(mae_test)
        mape_train_list.append(mape_train)
        mape_test_list.append(mape_test)

    # Calculate average metrics across all folds for this feature set
    feature_performance_tscv[f'{len(all_features)}_features'] = {
        'MSE_train': np.mean(mse_train_list),
        'MSE_test': np.mean(mse_test_list),
        'R²_train': np.mean(r2_train_list),
        'R²_test': np.mean(r2_test_list),
        'MAE_train': np.mean(mae_train_list),
        'MAE_test': np.mean(mae_test_list),
        'MAPE_train': np.mean(mape_train_list),
        'MAPE_test': np.mean(mape_test_list)
    }

    # Determine the least significant feature to remove based on the MSE test performance
    if len(all_features) > 1:
        last_feature_performance = feature_performance_tscv[f'{len(all_features)}_features']
        print(f"Performance with {len(all_features)} features: MSE Test: {last_feature_performance['MSE_test']}")
        
        # Fit the model with the current feature set to get coefficients
        model.fit(X_subset, y)
        coefficients = model.coef_
        
        # Create a DataFrame to store feature names and their coefficients
        coeff_df = pd.DataFrame({
            'Feature': all_features,
            'Coefficient': coefficients
        })
        
        # Print the coefficients
        print(coeff_df)
        
        # Plot the coefficients
        plt.figure(figsize=(10, 6))
        plt.barh(coeff_df['Feature'], coeff_df['Coefficient'], color='skyblue')
        plt.xlabel('Coefficient Value')
        plt.title('Feature Coefficients')
        plt.axvline(0, color='grey', linestyle='--')  # Vertical line at 0
        plt.show()
        
        # Drop the least significant feature
        all_features.remove(min(all_features, key=lambda f: last_feature_performance['MSE_test']))

    else:  # Exit the loop if only one feature remains
        print(f"Only one feature left: {all_features[0]}. Stopping feature removal.")
        break

# Step 5: Display performance for each feature set across all folds
for feature_count, metrics in feature_performance_tscv.items():
    print(f"Results for {feature_count}:")
    print(f"  MSE (Train): {metrics['MSE_train']}, MSE (Test): {metrics['MSE_test']}")
    print(f"  R² (Train): {metrics['R²_train']}, R² (Test): {metrics['R²_test']}")
    print(f"  MAE (Train): {metrics['MAE_train']}, MAE (Test): {metrics['MAE_test']}")
    print(f"  MAPE (Train): {metrics['MAPE_train']}, MAPE (Test): {metrics['MAPE_test']}")
    print("\n")


In [0]:
# Models for each horizon
horizons = [1, 2, 3]
results_overall = {}

# Create a new DataFrame to avoid modifying the original
df_horizon = df_aggregated_month_populated_btp_core.copy()

# Define a name for the DataFrame for model saving
df_name = "df_aggregated_month_populated_btp_core"

all_features = [
    'MONTHLY_CONTRACT_NET_VALUE_SUM',  
    'CONTRACT_DURATION_SUM', 
    'OVERCONSUMPTION_COUNT', 
    'ORDER_COUNT',  
    'TOTAL_CONSUMPTION_LAG_1'  # Keep only one lag feature to avoid redundancy
]

# Process each horizon
for h in horizons:
    print(f"\nProcessing horizon: {h} month(s) ahead")

    # Define the target variable for each horizon (shifted by -h)
    df_horizon[f'TOTAL_CONSUMPTION_{h}'] = df_horizon['TOTAL_CONSUMPTION_SUM'].shift(-h)

    # Only drop NaN values in the target variable, not the features
    df_horizon_filtered = df_horizon.dropna(subset=[f'TOTAL_CONSUMPTION_{h}'])


    # Define X (input features) to include all relevant features
    X = df_horizon_filtered[all_features]
    y = df_horizon_filtered[f'TOTAL_CONSUMPTION_{h}']

    # Initialize TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=5)  # Adjust n_splits as necessary
    
    # Initialize metrics storage for cross-validation
    metrics_per_fold = []

    # Ensure sorting by DATE and CUSTOMER_ID to maintain temporal order
    df_horizon_filtered.sort_values(by=['CUSTOMER_ID', 'DATE'], inplace=True)

    # Perform TimeSeriesSplit cross-validation
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Initialize Linear Regression model
        model = LinearRegression()

        # Fit the model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Calculate metrics for this fold
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)

        # Store metrics for the current fold
        metrics_per_fold.append({
            'MSE': mse,
            'R²': r2,
            'MAE': mae,
            'MAPE': mape
        })

    # Average the metrics across folds
    results_overall[h] = {
        'MSE': np.mean([m['MSE'] for m in metrics_per_fold]),
        'R²': np.mean([m['R²'] for m in metrics_per_fold]),
        'MAE': np.mean([m['MAE'] for m in metrics_per_fold]),
        'MAPE': np.mean([m['MAPE'] for m in metrics_per_fold])
    }
    
    # Save the model for the current horizon, incorporating the DataFrame name
    joblib.dump(model, f"linear_regression_model_{df_name}_horizon_{h}.pkl")
    print(f"Saved Linear Regression model for horizon {h} as 'linear_regression_model_{df_name}_horizon_{h}.pkl'.")

# Print overall results for each horizon
for h, metrics in results_overall.items():
    print(f"\nResults for horizon {h} month(s):")
    print(f"Mean MSE: {metrics['MSE']:.4f}")
    print(f"Mean R²: {metrics['R²']:.4f}")
    print(f"Mean MAE: {metrics['MAE']:.4f}")
    print(f"Mean MAPE: {metrics['MAPE']:.4f}%")


In [0]:
# Process each horizon and display feature importance
for h in horizons:
    print(f"\nProcessing horizon: {h} month(s) ahead")

    # Define the target variable for each horizon (shifted by -h)
    df_horizon[f'TOTAL_CONSUMPTION_{h}'] = df_horizon['TOTAL_CONSUMPTION_SUM'].shift(-h)

    # Only drop NaN values in the target variable, not the features
    df_horizon_filtered = df_horizon.dropna(subset=[f'TOTAL_CONSUMPTION_{h}'])

    # Ensure there are no unintended losses of rows due to feature NaNs for horizon 1
    if h == 1:
        print(f"Rows after dropping NaN for horizon {h}: {df_horizon_filtered.shape[0]}")

    # Define X (input features) to include all relevant features
    X = df_horizon_filtered[all_features]
    y = df_horizon_filtered[f'TOTAL_CONSUMPTION_{h}']

    # Initialize TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=5)  # Adjust n_splits as necessary
    
    # Initialize metrics storage for cross-validation
    metrics_per_fold = []

    # Ensure sorting by DATE and CUSTOMER_ID to maintain temporal order
    df_horizon_filtered.sort_values(by=['CUSTOMER_ID', 'DATE'], inplace=True)

    # Perform TimeSeriesSplit cross-validation
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Initialize Linear Regression model
        model = LinearRegression()

        # Fit the model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Calculate metrics for this fold
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)

        # Store metrics for the current fold
        metrics_per_fold.append({
            'MSE': mse,
            'R²': r2,
            'MAE': mae,
            'MAPE': mape
        })

    # Average the metrics across folds
    results_overall[h] = {
        'MSE': np.mean([m['MSE'] for m in metrics_per_fold]),
        'R²': np.mean([m['R²'] for m in metrics_per_fold]),
        'MAE': np.mean([m['MAE'] for m in metrics_per_fold]),
        'MAPE': np.mean([m['MAPE'] for m in metrics_per_fold])
    }

    # Extract feature importance (coefficients) after model is trained
    feature_importance = pd.Series(model.coef_, index=all_features)

    # Normalize to get relative importance
    feature_importance_normalized = abs(feature_importance) / sum(abs(feature_importance))

    # Display feature importance for this horizon
    print(f"\nFeature importance for horizon {h} month(s):")
    print(feature_importance_normalized)


Linear Regression for active contract Dataset

In [0]:
# Define the relative path to the Dataset folder
dataset_path = '../Datasets/df_filtered_active_customers.parquet'

# Read DataFrame from Parquet
df_filtered_active_customers = pd.read_parquet(dataset_path, engine='pyarrow')
print(df_filtered_active_customers)

In [0]:

# Step 1: Define the features and target variable
features = [
    'MONTHLY_CONTRACT_NET_VALUE_SUM',  
    'CONTRACT_DURATION_SUM', 'OVERCONSUMPTION_COUNT', 'ORDER_COUNT', 
    'ACTIVE_CONTRACT', 'TOTAL_CONSUMPTION_LAG_1'
]

X = df_filtered_active_customers[features].copy()  # Feature set
y = df_filtered_active_customers['TOTAL_CONSUMPTION_SUM']  # Target variable

# Step 2: Sort the DataFrame by DATE to maintain temporal order
df_filtered_active_customers.sort_values(by='DATE', inplace=True)

# Step 3: Apply target encoding to categorical columns
categorical_columns = ['ISS_TEXT', 'GLOBAL_REGION', 'COUNTRY', 'SAP_MASTER_CODE']
target_encoder = ce.TargetEncoder(cols=categorical_columns)
X_encoded = target_encoder.fit_transform(X, y)

# Step 4: Initialize TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)  # Adjust n_splits based on your preference

# Step 5: Train and evaluate a linear regression model using time series cross-validation
model = LinearRegression()

# Store metrics for each fold
mse_train_list, r2_train_list, mae_train_list, mape_train_list = [], [], [], []
mse_test_list, r2_test_list, mae_test_list, mape_test_list = [], [], [], []

for train_index, test_index in tscv.split(X_encoded):
    # Create training and test sets
    X_train, X_test = X_encoded.iloc[train_index], X_encoded.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions for both train and test sets
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculate metrics for the training set
    mse_train = mean_squared_error(y_train, y_train_pred)
    r2_train = r2_score(y_train, y_train_pred)
    mae_train = mean_absolute_error(y_train, y_train_pred)
    mape_train = mean_absolute_percentage_error(y_train.to_numpy(), y_train_pred)
    
    # Calculate metrics for the test set
    mse_test = mean_squared_error(y_test, y_test_pred)
    r2_test = r2_score(y_test, y_test_pred)
    mae_test = mean_absolute_error(y_test, y_test_pred)
    mape_test = mean_absolute_percentage_error(y_test.to_numpy(), y_test_pred)
    
    # Append results for each fold
    mse_train_list.append(mse_train)
    r2_train_list.append(r2_train)
    mae_train_list.append(mae_train)
    mape_train_list.append(mape_train)
    
    mse_test_list.append(mse_test)
    r2_test_list.append(r2_test)
    mae_test_list.append(mae_test)
    mape_test_list.append(mape_test)

# Step 6: Calculate the average and standard deviation of the metrics across all folds
average_metrics = {
    'MSE Train': np.mean(mse_train_list),
    'R² Train': np.mean(r2_train_list),
    'MAE Train': np.mean(mae_train_list),
    'MAPE Train': np.mean(mape_train_list),
    'MSE Test': np.mean(mse_test_list),
    'R² Test': np.mean(r2_test_list),
    'MAE Test': np.mean(mae_test_list),
    'MAPE Test': np.mean(mape_test_list)
}

std_metrics = {
    'MSE Train': np.std(mse_train_list),
    'R² Train': np.std(r2_train_list),
    'MAE Train': np.std(mae_train_list),
    'MAPE Train': np.std(mape_train_list),
    'MSE Test': np.std(mse_test_list),
    'R² Test': np.std(r2_test_list),
    'MAE Test': np.std(mae_test_list),
    'MAPE Test': np.std(mape_test_list)
}

# Print the averaged results across all folds
print("Averaged Results across all folds:")
for metric, value in average_metrics.items():
    print(f"{metric}: {value:.4f}")

# Optionally, print the standard deviation for a sense of variance across folds
print("\nStandard deviation across all folds:")
for metric, value in std_metrics.items():
    print(f"{metric}: {value:.4f}")


In [0]:
# Initialize lists to store predictions and actual values for both train and test sets
y_train_all, y_train_pred_all = [], []
y_test_all, y_test_pred_all = [], []

# Collect predictions for all folds
for train_index, test_index in tscv.split(X_encoded):
    # Create training and test sets
    X_train, X_test = X_encoded.iloc[train_index], X_encoded.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions for both train and test sets
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Append predictions and actual values
    y_train_all.extend(y_train)
    y_train_pred_all.extend(y_train_pred)
    y_test_all.extend(y_test)
    y_test_pred_all.extend(y_test_pred)

# Convert lists to numpy arrays for easier manipulation
y_train_all = np.array(y_train_all)
y_train_pred_all = np.array(y_train_pred_all)
y_test_all = np.array(y_test_all)
y_test_pred_all = np.array(y_test_pred_all)

# Step 1: Scatter Plot of Predicted vs Actual Values for Training Set
plt.figure(figsize=(14, 6))

# Training set
plt.subplot(1, 2, 1)
sns.scatterplot(x=y_train_all, y=y_train_pred_all, color='blue', alpha=0.5)
plt.plot([y_train_all.min(), y_train_all.max()], [y_train_all.min(), y_train_all.max()], 'r--')  # 45-degree line
plt.title('Training Set: Predicted vs Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.axis('equal')

# Step 2: Scatter Plot of Predicted vs Actual Values for Testing Set
plt.subplot(1, 2, 2)
sns.scatterplot(x=y_test_all, y=y_test_pred_all, color='green', alpha=0.5)
plt.plot([y_test_all.min(), y_test_all.max()], [y_test_all.min(), y_test_all.max()], 'r--')  # 45-degree line
plt.title('Testing Set: Predicted vs Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.axis('equal')

plt.tight_layout()
plt.show()

# Step 3: Residuals Plot for Testing Set
residuals = y_test_all - y_test_pred_all

plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test_pred_all, y=residuals, color='purple', alpha=0.6)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals Plot')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.show()


In [0]:
# Model for each horizon
horizons = [1, 2, 3]
results_overall = {}

# Create a new DataFrame to avoid modifying the original
df_horizon = df_filtered_active_customers.copy()

# Define a name for the DataFrame for model saving
df_name = "df_filtered_active_customers"

all_features = [
    'MONTHLY_CONTRACT_NET_VALUE_SUM',  
    'CONTRACT_DURATION_SUM', 
    'OVERCONSUMPTION_COUNT', 
    'ORDER_COUNT',  
    'TOTAL_CONSUMPTION_LAG_1'  # Keep only one lag feature to avoid redundancy
]

# Process each horizon
for h in horizons:
    print(f"\nProcessing horizon: {h} month(s) ahead")

    # Define the target variable for each horizon (shifted by -h)
    df_horizon[f'TOTAL_CONSUMPTION_{h}'] = df_horizon['TOTAL_CONSUMPTION_SUM'].shift(-h)

    # Only drop NaN values in the target variable, not the features
    df_horizon_filtered = df_horizon.dropna(subset=[f'TOTAL_CONSUMPTION_{h}'])

    # Ensure there are no unintended losses of rows due to feature NaNs for horizon 1
    if h == 1:
        print(f"Rows after dropping NaN for horizon {h}: {df_horizon_filtered.shape[0]}")

    # Define X (input features) to include all relevant features
    X = df_horizon_filtered[all_features]
    y = df_horizon_filtered[f'TOTAL_CONSUMPTION_{h}']

    # Initialize TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=5)  # Adjust n_splits as necessary
    
    # Initialize metrics storage for cross-validation
    metrics_per_fold = []

    # Ensure sorting by DATE and CUSTOMER_ID to maintain temporal order
    df_horizon_filtered.sort_values(by=['CUSTOMER_ID', 'DATE'], inplace=True)

    # Perform TimeSeriesSplit cross-validation
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Initialize Linear Regression model
        model = LinearRegression()

        # Fit the model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Calculate metrics for this fold
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)

        # Store metrics for the current fold
        metrics_per_fold.append({
            'MSE': mse,
            'R²': r2,
            'MAE': mae,
            'MAPE': mape
        })

    # Average the metrics across folds
    results_overall[h] = {
        'MSE': np.mean([m['MSE'] for m in metrics_per_fold]),
        'R²': np.mean([m['R²'] for m in metrics_per_fold]),
        'MAE': np.mean([m['MAE'] for m in metrics_per_fold]),
        'MAPE': np.mean([m['MAPE'] for m in metrics_per_fold])
    }
    
    # Save the model for the current horizon, incorporating the DataFrame name
    joblib.dump(model, f"linear_regression_model_{df_name}_horizon_{h}.pkl")
    print(f"Saved Linear Regression model for horizon {h} as 'linear_regression_model_{df_name}_horizon_{h}.pkl'.")

# Print overall results for each horizon
for h, metrics in results_overall.items():
    print(f"\nResults for horizon {h} month(s):")
    print(f"Mean MSE: {metrics['MSE']:.4f}")
    print(f"Mean R²: {metrics['R²']:.4f}")
    print(f"Mean MAE: {metrics['MAE']:.4f}")
    print(f"Mean MAPE: {metrics['MAPE']:.4f}%")


In [0]:
# Process each horizon and display feature importance
for h in horizons:
    print(f"\nProcessing horizon: {h} month(s) ahead")

    # Define the target variable for each horizon (shifted by -h)
    df_horizon[f'TOTAL_CONSUMPTION_{h}'] = df_horizon['TOTAL_CONSUMPTION_SUM'].shift(-h)

    # Only drop NaN values in the target variable, not the features
    df_horizon_filtered = df_horizon.dropna(subset=[f'TOTAL_CONSUMPTION_{h}'])


    # Define X (input features) to include all relevant features
    X = df_horizon_filtered[all_features]
    y = df_horizon_filtered[f'TOTAL_CONSUMPTION_{h}']

    # Initialize TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=5)  # Adjust n_splits as necessary
    
    # Initialize metrics storage for cross-validation
    metrics_per_fold = []

    # Ensure sorting by DATE and CUSTOMER_ID to maintain temporal order
    df_horizon_filtered.sort_values(by=['CUSTOMER_ID', 'DATE'], inplace=True)

    # Perform TimeSeriesSplit cross-validation
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Initialize Linear Regression model
        model = LinearRegression()

        # Fit the model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Calculate metrics for this fold
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)

        # Store metrics for the current fold
        metrics_per_fold.append({
            'MSE': mse,
            'R²': r2,
            'MAE': mae,
            'MAPE': mape
        })

    # Average the metrics across folds
    results_overall[h] = {
        'MSE': np.mean([m['MSE'] for m in metrics_per_fold]),
        'R²': np.mean([m['R²'] for m in metrics_per_fold]),
        'MAE': np.mean([m['MAE'] for m in metrics_per_fold]),
        'MAPE': np.mean([m['MAPE'] for m in metrics_per_fold])
    }

    # Extract feature importance (coefficients) after model is trained
    feature_importance = pd.Series(model.coef_, index=all_features)

    # Normalize to get relative importance
    feature_importance_normalized = abs(feature_importance) / sum(abs(feature_importance))

    # Display feature importance for this horizon
    print(f"\nFeature importance for horizon {h} month(s):")
    print(feature_importance_normalized)


